In [ ]:
import numpy as np, itertools as it, math
from tqdm import tqdm
from functools import cache

## utils

In [ ]:
from cpd_search import *  # using cpd/original/cpd_search.py

In [ ]:
def all_tensors(shape,MOD):
    return (
        np.array(v,dtype=int).reshape(shape)
        for v in it.product(range(MOD),repeat=np.prod(shape))
    )

In [ ]:
def all_invle_mats(n,MOD):
    for v in it.product(range(MOD),repeat=n*n):
        M=np.array(v,dtype=int).reshape((n,n))
        if mat_rank(M,MOD)==n:
            yield M

In [ ]:
def tensor2code(T):
    '''
    convert tensor T into a hashable object
    '''
    assert isinstance(T,np.ndarray), type(T)
    return (T.shape,tuple(T.flatten()))

In [ ]:
def linsys(A,B,MOD,iterate=True):
    '''
    if iter:
        return iterable of all solutions M to A@M=B mod MOD
    else:
        return X,Y such that the set {X+Y@Z : Z}
        parameterizes all such solutions
    '''
    assert len(A.shape)==2, A.shape
    assert len(B.shape)==2, B.shape
    assert A.shape[0]==B.shape[0], (A.shape,B.shape)
    rrefA,P,_,r=mat_row_reduce(A,MOD)
    _,QT,_,_=mat_row_reduce(rrefA.T,MOD)
    Q=QT.T
    reduced_expected=np.zeros(A.shape,dtype=int)
    reduced_expected[range(r),range(r)]=1
    assert np.array_equal((P@A@Q)%MOD,reduced_expected), (P,A,Q,r,(P@A@Q)%MOD)
    # (P@A@Q)@(inv(Q)@M)=P@B
    PB=(P@B)%MOD
    if not (PB[r:,:]==0).all():
        # no solutions
        if iterate:
            return []
        else:
            return None
    # M = Q@cat([PB[:r,:],arbitrary],axis=0)
    #   = Q[:,:r]@PB[:r,:] + Q[:,r:]@arbitrary
    M0,M1=(Q[:,:r]@PB[:r,:])%MOD, Q[:,r:]
    if iterate:
        return (
            (M0+M1@X)%MOD
            for X in all_tensors((M1.shape[1],M0.shape[1]),MOD)
        )
    else:
        return M0,M1

In [ ]:
def test_linsys(A,B,MOD):
    assert len(A.shape)==2, f'{A.shape=}'
    assert len(B.shape)==2, f'{B.shape=}'
    assert A.shape[0]==B.shape[0], f'{(A.shape,B.shape)=}'
    expected_sols=sorted(
        tensor2code(M)
        for M in all_tensors((A.shape[1],B.shape[1]),MOD)
        if np.array_equal((A@M)%MOD,B)
    )
    
    actual_sols=sorted(
        tensor2code(M)
        for M in linsys(A,B,MOD,iterate=True)
    )
    assert expected_sols==actual_sols
    
#     if ret is None:
#         assert len(expected_sols)==0, 'erroneously detected system as having no solutions'
#     else:
#         M0,M1=ret
#         expected_M_shape=(A.shape[1],B.shape[1])
#         assert M0.shape==expected_M_shape, f'{(expected_M_shape,M0.shape)=}'
#         assert M1.shape[0]==A.shape[1], f'{(A.shape,M1.shape)=}'
#         actual_sols=sorted(
#             tensor2code((M0+M1@X)%MOD)
#             for X in all_tensors((M1.shape[1],B.shape[1]),MOD)
#         )

In [ ]:
for A,B,MOD in [
    # degenerate systems
    (np.ones((0,0),dtype=int),np.zeros((0,0),dtype=int),2),
    (np.ones((1,0),dtype=int),np.zeros((1,1),dtype=int),2),
    
    # no solution
    (np.zeros((1,1),dtype=int),np.ones((1,1),dtype=int),2),
    
    # solution is an affine set not touching the origin
    (np.array([[1,1],[1,0]],dtype=int),np.array([[1,1],[2,1]],dtype=int),3),
    
    # A full-row-rank but not full-clm-rank
    (np.array([[1,1,0],[1,0,1]],dtype=int),np.array([[1,1],[2,1]],dtype=int),3),
    
    # A full-clm-rank but not full-row-rank
    (np.array([[1,1,0],[1,0,1]],dtype=int).T,np.array([[0],[1],[2]],dtype=int),3),
]:
#     display(linsys(A,B,MOD))
    test_linsys(A,B,MOD)

## generating tensors

In [ ]:
def dfs_canonicals_dim3(
    n_slices,nrows,nclms,MOD,row_ops,
    cur_slices,sol_list
):
    '''
    row_ops: list of allowed row operations on slices
    cur_slices: current list (prefix) of slices in tensor that will be built
    sol_list (mutated): list to append to
    '''
    assert isinstance(row_ops,list)
    assert len(cur_slices)<=n_slices
    assert all(M.shape==(nrows,nclms) for M in cur_slices)
    
    if len(cur_slices)==n_slices:
        sol_list.append(np.array(cur_slices,dtype=int))
        return
    
    print('='*32+'\n'+f'depth: {len(cur_slices)}')
    good_pairs=[]
    for P in tqdm(row_ops):
        # find all inv'le nclms x nclms Q s.t. all(M==P@M@Q for M in cur_slices)
        sys_left=np.concatenate([
            (P@M)%MOD
            for M in cur_slices
        ],axis=0)
        sys_right=np.concatenate(cur_slices,axis=0)
        good_pairs.extend(
            (P,Q)
            for Q in linsys(sys_left,sys_right,MOD)
            if mat_rank(Q,MOD)==nclms
        )
            
    assert len(good_pairs)>0  # should at least contain the identity
    print(f'# good pairs={len(good_pairs)}')
    
    seen=set()
    canonicals=[]
    for M in all_tensors((nrows,nclms),MOD):
        code=tensor2code(M)
        if code not in seen:
            canonicals.append(M)
            seen.update(
                tensor2code((P@M@Q)%MOD)
                for P,Q in good_pairs
            )
    print(f'# canonicals={len(canonicals)}')
    
    for M in canonicals:
        dfs_canonicals_dim3(
            n_slices,nrows,nclms,MOD,row_ops,
            cur_slices+[M,],sol_list
        )

### 3x4x3

In [ ]:
canonical_tensors=[]
A=np.zeros((4,3),dtype=int)
A[range(3),range(3)]=1
init_slices=[
    A,
]
display(init_slices)
dfs_canonicals_dim3(
    n_slices=3,nrows=4,nclms=3,MOD=2,
    row_ops=list(all_invle_mats(4,2)),
    cur_slices=init_slices,
    sol_list=canonical_tensors
)

In [ ]:
len(canonical_tensors)

In [ ]:
with open('canonical-3x4x3-first-slice-rank-3.txt','w') as f:
    for T in canonical_tensors:
        assert T.shape==(3,4,3), (T.shape,T)
        f.write(' '.join(str(v) for v in T.flatten())+'\n')

### 3x4x4

In [ ]:
canonical_tensors=[]
for r_init in [3,4]:
    A=np.zeros((4,4),dtype=int)
    A[range(r_init),range(r_init)]=1
    init_slices=[
        A,
    ]
    print('='*64)
    display(init_slices)
    dfs_canonicals_dim3(
        n_slices=3,nrows=4,nclms=4,MOD=2,
        row_ops=list(all_invle_mats(4,2)),
        cur_slices=init_slices,
        sol_list=canonical_tensors
    )

In [ ]:
len(canonical_tensors)

In [ ]:
with open('canonical-3x4x4-first-slice-rank-ge3.txt','w') as f:
    for T in canonical_tensors:
        assert T.shape==(3,4,4), (T.shape,T)
        f.write(' '.join(str(v) for v in T.flatten())+'\n')

### 3x3x5

In [ ]:
canonical_tensors=[]
A=np.zeros((3,5),dtype=int)
A[range(3),range(3)]=1
init_slices=[
    A,
]
display(init_slices)
dfs_canonicals_dim3(
    n_slices=3,nrows=3,nclms=5,MOD=2,
    row_ops=list(all_invle_mats(3,2)),
    cur_slices=init_slices,
    sol_list=canonical_tensors
)

In [ ]:
len(canonical_tensors)

In [ ]:
with open('canonical-3x3x5-first-slice-rank-3.txt','w') as f:
    for T in canonical_tensors:
        assert T.shape==(3,3,5), (T.shape,T)
        f.write(' '.join(str(v) for v in T.flatten())+'\n')

## $\newcommand{\F}{\mathbb{F}}$maximum rank results over $\F_2$ (Java)
* $R_{\F_2}(3,4,3)=6$ <span style="color:gray">$=R_{\F_2}(3,3,4)$</span>
* $R_{\F_2}(3,4,4)=8$
* $R_{\F_2}(3,3,5)=7$